이 연습에선 스파크 MLlib 및 ALS를 사용해 권장 사항을 구현하는 방법을 보여 준다.

In [1]:
import os
SPARK_HOME = '/Users/isang-geon/spark'
# exec(open(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py')).read())
exec(open(os.path.join(SPARK_HOME, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.0.1
      /_/

Using Python version 3.8.5 (default, Sep  4 2020 02:22:02)
SparkSession available as 'spark'.


In [2]:
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

In [9]:
# data = sc.textFile("hdfs://" + SPARK_HOME + '/data/movielens')
data = sc.textFile("file://" + SPARK_HOME + '/data/movielens')
data.take(5)

['196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596']

In [13]:
ratings = data.map(lambda x: x.split('\t')) \
                .map(lambda x: Rating(int(x[0]), int(x[1]), float(x[2])))
ratings.take(5)

[Rating(user=196, product=242, rating=3.0),
 Rating(user=186, product=302, rating=3.0),
 Rating(user=22, product=377, rating=1.0),
 Rating(user=244, product=51, rating=2.0),
 Rating(user=166, product=346, rating=1.0)]

In [14]:
rank = 10
numIterations = 10
model = ALS.train(ratings, rank, numIterations)

In [17]:
testdata = ratings.map(lambda p:(p[0], p[1]))
predictions = model.predictAll(testdata) \
                    .map(lambda r:((r[0], r[1]), r[2]))
predictions.take(10)

[((504, 384), 2.35451735394139),
 ((648, 384), 3.4580079375549815),
 ((64, 384), 2.5997184727666487),
 ((472, 384), 3.283587108774022),
 ((896, 384), 2.692119500799067),
 ((280, 384), 3.8359512753882603),
 ((880, 384), 2.856541666576656),
 ((417, 384), 3.072823783352962),
 ((497, 384), 2.441328644897369),
 ((545, 384), 2.700306518746582)]

In [18]:
ratesAndPreds = ratings.map(lambda r:((r[0], r[1]), r[2])) \
                        .join(predictions)
ratesAndPreds.take(10)

[((196, 242), (3.0, 3.967824526123883)),
 ((186, 302), (3.0, 3.113127047039603)),
 ((305, 451), (3.0, 2.8830778660854843)),
 ((210, 40), (3.0, 2.8535187716970687)),
 ((234, 1184), (2.0, 1.6097413502066225)),
 ((160, 234), (5.0, 4.353214917053813)),
 ((225, 193), (4.0, 4.667482579999463)),
 ((290, 88), (4.0, 3.8353785339024182)),
 ((181, 1081), (1.0, 1.0692081091065813)),
 ((20, 288), (1.0, 1.9739580452399301))]

In [16]:
MSE = ratesAndPreds.map(lambda r:(r[1][0] - r[1][1]) ** 2) \
                    .mean()
print("Mean Squared Error = " + str(MSE))

Mean Squared Error = 0.48302452821148517


In [19]:
model.save(sc, "ratings_model")

In [20]:
from pyspark.mllib.recommendation import MatrixFactorizationModel
reloaded_model = MatrixFactorizationModel.load(sc, "ratings_model")
reloaded_model